In [14]:
import speech_recognition as sr

In [15]:
r = sr.Recognizer()

In [16]:
with sr.Microphone() as source:
    print("Say something")
    audio = r.listen(source)
    print("Time over, thanks")

try:
    print("TEXT: " + r.recognize_google(audio, language="pt-bt",)
except:
    pass

Say something
Time over, thanks
TEXT: O Lucas é um c****


In [4]:
import requests
import json
import pandas as pd

In [5]:
# initialize variables
auth_url = "https://api.hatebase.org/4-2/authenticate"
vocab_url = "https://api.hatebase.org/4-2/get_vocabulary"
lang = "por"
resp_format = "json"
token = ""
pages = 0
total_entries = 0

In [6]:
# initialize authentication payload
auth_payload = "api_key=ZJmEGTDmnEUvkZVMZHNQcgwzRxuMZaNc"
headers = {
    'Content-Type': "application/x-www-form-urlencoded",
    'cache-control': "no-cache"
    }

In [7]:
# authenticate
auth_resp = requests.request("POST", auth_url, data=auth_payload, headers=headers)
print(auth_resp)

<Response [200]>


In [8]:
# get the session token
token = auth_resp.json()["result"]["token"]
# quick check if it worked
print(auth_resp.text, token)

{
    "datetime": "2019-06-19 03:35:32",
    "terms": "Your use of the Hatebase API acknowledges your consent with Hatebase's Terms of Use (hatebase.org\/terms) and prohibits redistribution of the data herein to any third-parties for any purpose, including but not limited to republication in any form, such as in public code repositories.",
    "api_key": "ZJmEGTDmnEUvkZVMZHNQcgwzRxuMZaNc",
    "result": {
        "token": "iDhsNBXKbVgMfMryTHGKDqETHwTAuNhB",
        "expires_on": "2019-06-19 04:35:32"
    }
} iDhsNBXKbVgMfMryTHGKDqETHwTAuNhB


In [9]:
# initialize vocabulary payload
# first without any given page-number
vocab_payload = "token=" + token + "&format=" + resp_format + "&language=" + lang
voc_resp = requests.request("POST", vocab_url, data=vocab_payload, headers=headers)
voc_json = voc_resp.json()

In [10]:
# how many pages in total? and how many results= entries in the vocab?
pages = voc_json["number_of_pages"]
results = voc_json["number_of_results"]

In [11]:
# create vocabulary df from first resultset
df_voc = pd.DataFrame(voc_json["result"])

In [12]:
# now get results of all remaining pages and append to df_voc
for page in range(2,pages+1):
    vocab_payload = "token=" + token + "&format=json&language=" + lang + "&page=" + str(page)
    voc_resp = requests.request("POST", vocab_url, data=vocab_payload, headers=headers)
    voc_json = voc_resp.json()
    df_voc = df_voc.append(voc_json["result"])

# reset df_voc index
df_voc.reset_index(drop=True, inplace=True)

In [13]:
df_voc.to_csv("hatebase_vocab.csv")